In [4]:
import platform
import os
import subprocess
import re

def get_hardware_info():
    """Gather information about the system hardware"""
    hardware_info = {}
    
    # Basic system information
    hardware_info["system"] = platform.system()
    hardware_info["node"] = platform.node()
    hardware_info["release"] = platform.release()
    hardware_info["version"] = platform.version()
    hardware_info["machine"] = platform.machine()
    hardware_info["processor"] = platform.processor()
    
    # CPU information
    if hardware_info["system"] == "Linux":
        try:
            with open("/proc/cpuinfo", "r") as f:
                cpuinfo = f.read()
            
            # Extract CPU model
            model_name = re.search(r"model name\s+:\s+(.*)", cpuinfo)
            if model_name:
                hardware_info["cpu_model"] = model_name.group(1)
            
            # Count CPU cores
            cores = len(re.findall(r"processor\s+:", cpuinfo))
            hardware_info["cpu_cores"] = cores
        except:
            hardware_info["cpu_model"] = "Unknown"
            hardware_info["cpu_cores"] = "Unknown"
    
    elif hardware_info["system"] == "Darwin":  # macOS
        try:
            cpu_model = subprocess.check_output(["sysctl", "-n", "machdep.cpu.brand_string"]).decode().strip()
            hardware_info["cpu_model"] = cpu_model
            
            cpu_cores = subprocess.check_output(["sysctl", "-n", "hw.physicalcpu"]).decode().strip()
            hardware_info["cpu_cores"] = cpu_cores
        except:
            hardware_info["cpu_model"] = "Unknown"
            hardware_info["cpu_cores"] = "Unknown"
    
    elif hardware_info["system"] == "Windows":
        try:
            cpu_model = subprocess.check_output("wmic cpu get name", shell=True).decode().strip().split('\n')[1]
            hardware_info["cpu_model"] = cpu_model
            
            cpu_cores = subprocess.check_output("wmic cpu get NumberOfCores", shell=True).decode().strip().split('\n')[1]
            hardware_info["cpu_cores"] = cpu_cores
        except:
            hardware_info["cpu_model"] = "Unknown"
            hardware_info["cpu_cores"] = "Unknown"
    
    # Memory information
    if hardware_info["system"] == "Linux":
        try:
            with open("/proc/meminfo", "r") as f:
                meminfo = f.read()
            
            total_memory = re.search(r"MemTotal:\s+(\d+)", meminfo)
            if total_memory:
                hardware_info["total_memory"] = f"{int(total_memory.group(1)) // 1024} MB"
        except:
            hardware_info["total_memory"] = "Unknown"
    
    elif hardware_info["system"] == "Darwin":  # macOS
        try:
            total_memory = subprocess.check_output(["sysctl", "-n", "hw.memsize"]).decode().strip()
            hardware_info["total_memory"] = f"{int(total_memory) // (1024 * 1024)} MB"
        except:
            hardware_info["total_memory"] = "Unknown"
    
    elif hardware_info["system"] == "Windows":
        try:
            total_memory = subprocess.check_output("wmic ComputerSystem get TotalPhysicalMemory", shell=True).decode().strip().split('\n')[1]
            hardware_info["total_memory"] = f"{int(total_memory) // (1024 * 1024)} MB"
        except:
            hardware_info["total_memory"] = "Unknown"
    
    # GPU information
    if hardware_info["system"] == "Linux":
        try:
            gpu_info = subprocess.check_output("lspci | grep -i 'vga\\|3d\\|2d'", shell=True).decode()
            if gpu_info:
                hardware_info["gpu"] = gpu_info.strip()
            else:
                hardware_info["gpu"] = "No GPU detected"
        except:
            hardware_info["gpu"] = "Unknown"
    
    elif hardware_info["system"] == "Darwin":  # macOS
        try:
            gpu_info = subprocess.check_output("system_profiler SPDisplaysDataType | grep Chipset", shell=True).decode()
            if gpu_info:
                hardware_info["gpu"] = gpu_info.strip()
            else:
                hardware_info["gpu"] = "No GPU detected"
        except:
            hardware_info["gpu"] = "Unknown"
    
    elif hardware_info["system"] == "Windows":
        try:
            gpu_info = subprocess.check_output("wmic path win32_VideoController get name", shell=True).decode()
            if gpu_info:
                hardware_info["gpu"] = gpu_info.split('\n')[1].strip()
            else:
                hardware_info["gpu"] = "No GPU detected"
        except:
            hardware_info["gpu"] = "Unknown"
    
    # Disk information
    if hardware_info["system"] == "Linux":
        try:
            disk_info = subprocess.check_output("df -h /", shell=True).decode()
            hardware_info["disk"] = disk_info.strip()
        except:
            hardware_info["disk"] = "Unknown"
    
    elif hardware_info["system"] == "Darwin" or hardware_info["system"] == "Windows":
        try:
            disk_info = subprocess.check_output("df -h /", shell=True).decode()
            hardware_info["disk"] = disk_info.strip()
        except:
            try:
                # Alternative for Windows
                disk_info = subprocess.check_output("wmic logicaldisk get deviceid, size, freespace", shell=True).decode()
                hardware_info["disk"] = disk_info.strip()
            except:
                hardware_info["disk"] = "Unknown"
    
    return hardware_info

def print_hardware_info():
    """Print the hardware information in a readable format"""
    info = get_hardware_info()
    
    print("\n===== HARDWARE INFORMATION =====")
    print(f"System: {info.get('system', 'Unknown')}")
    print(f"Hostname: {info.get('node', 'Unknown')}")
    print(f"OS Version: {info.get('release', 'Unknown')} {info.get('version', 'Unknown')}")
    print(f"Architecture: {info.get('machine', 'Unknown')}")
    print(f"Processor: {info.get('processor', 'Unknown')}")
    print(f"CPU Model: {info.get('cpu_model', 'Unknown')}")
    print(f"CPU Cores: {info.get('cpu_cores', 'Unknown')}")
    print(f"Total Memory: {info.get('total_memory', 'Unknown')}")
    print(f"GPU: {info.get('gpu', 'Unknown')}")
    print("\nDisk Information:")
    print(info.get('disk', 'Unknown'))
    print("===============================")

if __name__ == "__main__":
    print_hardware_info()


===== HARDWARE INFORMATION =====
System: Linux
Hostname: fd97cee3f414
OS Version: 5.15.0-134-generic #145-Ubuntu SMP Wed Feb 12 20:08:39 UTC 2025
Architecture: x86_64
Processor: x86_64
CPU Model: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
CPU Cores: 112
Total Memory: 257358 MB
GPU: Unknown

Disk Information:
Filesystem      Size  Used Avail Use% Mounted on
overlay         2.2T  716G  1.4T  35% /


/bin/sh: 1: lspci: not found


In [3]:
print("Available GPUs:")
for i in range(torch.cuda.device_count()):
    print(f" - GPU {i}: {torch.cuda.get_device_name(i)}")


Available GPUs:
 - GPU 0: NVIDIA A10
 - GPU 1: NVIDIA A10


In [5]:
import torch
print("hi")
device = torch.device( "cuda:0" if torch.cuda.is_available() else "cpu")
print ( device)

hi
cuda:0


In [6]:
!nvidia-smi


Mon Apr 14 18:31:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10                     Off |   00000000:17:00.0 Off |                    0 |
|  0%   30C    P8             16W /  150W |       4MiB /  23028MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
import torch
torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.get_device_name(0)


(True, 2, 'NVIDIA A10')

In [8]:
import psutil
import platform
import subprocess
import re
import time
from datetime import datetime

def get_cpu_memory_gb():
    """Get CPU memory information in GB"""
    # Get virtual memory statistics
    memory = psutil.virtual_memory()
    
    # Calculate values in GB (1 GB = 1024^3 bytes)
    total_memory_gb = memory.total / (1024 ** 3)
    used_memory_gb = memory.used / (1024 ** 3)
    available_memory_gb = memory.available / (1024 ** 3)
    
    return {
        "total_gb": round(total_memory_gb, 2),
        "used_gb": round(used_memory_gb, 2),
        "available_gb": round(available_memory_gb, 2),
        "percent_used": memory.percent
    }

def get_gpu_memory_info_gb():
    """Get GPU memory information in GB based on the platform"""
    system = platform.system()
    gpu_info = []
    
    if system == "Windows":
        try:
            # For NVIDIA GPUs on Windows
            nvidia_smi_output = subprocess.check_output(
                ['nvidia-smi', '--query-gpu=name,memory.total,memory.used,memory.free', '--format=csv,noheader,nounits'],
                universal_newlines=True
            )
            
            for line in nvidia_smi_output.strip().split('\n'):
                name, total, used, free = [item.strip() for item in line.split(',')]
                gpu_info.append({
                    "name": name,
                    "total_gb": round(float(total) / 1024, 2),  # Convert from MB to GB
                    "used_gb": round(float(used) / 1024, 2),
                    "free_gb": round(float(free) / 1024, 2)
                })
                
        except (subprocess.SubprocessError, FileNotFoundError):
            gpu_info.append({"error": "NVIDIA tools not available or no NVIDIA GPU found"})
            
    elif system == "Linux":
        try:
            # For NVIDIA GPUs on Linux
            nvidia_smi_output = subprocess.check_output(
                ['nvidia-smi', '--query-gpu=name,memory.total,memory.used,memory.free', '--format=csv,noheader,nounits'],
                universal_newlines=True
            )
            
            for line in nvidia_smi_output.strip().split('\n'):
                name, total, used, free = [item.strip() for item in line.split(',')]
                gpu_info.append({
                    "name": name,
                    "total_gb": round(float(total) / 1024, 2),  # Convert from MB to GB
                    "used_gb": round(float(used) / 1024, 2),
                    "free_gb": round(float(free) / 1024, 2)
                })
                
        except (subprocess.SubprocessError, FileNotFoundError):
            # Try AMD GPUs on Linux
            try:
                rocm_smi_output = subprocess.check_output(['rocm-smi', '--showmeminfo', 'vram'], universal_newlines=True)
                pattern = r"GPU\[(\d+)\].*?:\s+(\d+)\s+(\d+)"
                matches = re.findall(pattern, rocm_smi_output)
                
                for gpu_id, used, total in matches:
                    gpu_info.append({
                        "name": f"AMD GPU {gpu_id}",
                        "total_gb": round(float(total) / (1024 ** 2), 2),  # Convert from KB to GB
                        "used_gb": round(float(used) / (1024 ** 2), 2),
                        "free_gb": round((float(total) - float(used)) / (1024 ** 2), 2)
                    })
                    
            except (subprocess.SubprocessError, FileNotFoundError):
                gpu_info.append({"error": "No GPU tools available or no supported GPU found"})
                
    elif system == "Darwin":  # macOS
        try:
            # This is a simplified approach for macOS - may not work on all systems
            system_profiler = subprocess.check_output(['system_profiler', 'SPDisplaysDataType'], universal_newlines=True)
            
            # Extract GPU names
            gpu_names = re.findall(r"Chipset Model: (.+)", system_profiler)
            
            # Extract VRAM if available
            vram_matches = re.findall(r"VRAM \(Total\): (\d+) MB", system_profiler)
            
            for i, name in enumerate(gpu_names):
                gpu_data = {"name": name}
                
                if i < len(vram_matches):
                    total_mb = int(vram_matches[i])
                    gpu_data["total_gb"] = round(total_mb / 1024, 2)
                    gpu_data["note"] = "macOS doesn't provide detailed GPU memory usage information"
                else:
                    gpu_data["note"] = "VRAM information not available"
                    
                gpu_info.append(gpu_data)
                
        except (subprocess.SubprocessError, FileNotFoundError):
            gpu_info.append({"error": "Could not retrieve GPU information on macOS"})
    else:
        gpu_info.append({"error": f"Unsupported operating system: {system}"})
        
    return gpu_info

def monitor_resources(interval=1, duration=10):
    """Monitor CPU and GPU resources for a specified duration and interval"""
    iterations = int(duration / interval)
    
    print(f"\n{'=' * 60}")
    print(f"RESOURCE MONITOR - Starting at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Monitoring for {duration} seconds with {interval} second intervals")
    print(f"{'=' * 60}")
    
    for i in range(iterations):
        # Clear screen for better visibility in terminal (comment out if not desired)
        # os.system('cls' if platform.system() == 'Windows' else 'clear')
        
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print(f"\n[{timestamp}] - Iteration {i+1}/{iterations}")
        
        # Get CPU memory info
        cpu_memory = get_cpu_memory_gb()
        print("\nCPU MEMORY:")
        print(f"  Total: {cpu_memory['total_gb']:.2f} GB")
        print(f"  Used: {cpu_memory['used_gb']:.2f} GB ({cpu_memory['percent_used']}%)")
        print(f"  Available: {cpu_memory['available_gb']:.2f} GB")
        
        # Get GPU memory info
        gpu_memory_info = get_gpu_memory_info_gb()
        print("\nGPU MEMORY:")
        
        if not gpu_memory_info:
            print("  No GPU information available")
        else:
            for idx, gpu in enumerate(gpu_memory_info):
                print(f"  GPU {idx+1}:")
                
                if "error" in gpu:
                    print(f"    {gpu['error']}")
                elif "note" in gpu:
                    print(f"    Name: {gpu['name']}")
                    if "total_gb" in gpu:
                        print(f"    Total Memory: {gpu['total_gb']:.2f} GB")
                    print(f"    Note: {gpu['note']}")
                else:
                    print(f"    Name: {gpu['name']}")
                    print(f"    Total Memory: {gpu['total_gb']:.2f} GB")
                    print(f"    Used Memory: {gpu['used_gb']:.2f} GB")
                    print(f"    Free Memory: {gpu['free_gb']:.2f} GB")
                    usage_percent = (gpu['used_gb'] / gpu['total_gb']) * 100
                    print(f"    Usage: {usage_percent:.1f}%")
        
        print(f"\n{'-' * 60}")
        
        # Wait for the next iteration, unless it's the last one
        if i < iterations - 1:
            time.sleep(interval)

if __name__ == "__main__":
    # You can customize the monitoring duration and interval
    monitor_resources(interval=2, duration=10)  # Check every 2 seconds for 10 seconds


RESOURCE MONITOR - Starting at 2025-04-14 18:31:32
Monitoring for 10 seconds with 2 second intervals

[2025-04-14 18:31:32] - Iteration 1/5

CPU MEMORY:
  Total: 251.33 GB
  Used: 8.78 GB (4.2%)
  Available: 240.71 GB

GPU MEMORY:
  GPU 1:
    Name: NVIDIA A10
    Total Memory: 22.49 GB
    Used Memory: 0.00 GB
    Free Memory: 22.09 GB
    Usage: 0.0%
  GPU 2:
    Name: NVIDIA A10
    Total Memory: 22.49 GB
    Used Memory: 0.00 GB
    Free Memory: 22.09 GB
    Usage: 0.0%

------------------------------------------------------------

[2025-04-14 18:31:34] - Iteration 2/5

CPU MEMORY:
  Total: 251.33 GB
  Used: 8.78 GB (4.2%)
  Available: 240.71 GB

GPU MEMORY:
  GPU 1:
    Name: NVIDIA A10
    Total Memory: 22.49 GB
    Used Memory: 0.00 GB
    Free Memory: 22.09 GB
    Usage: 0.0%
  GPU 2:
    Name: NVIDIA A10
    Total Memory: 22.49 GB
    Used Memory: 0.00 GB
    Free Memory: 22.09 GB
    Usage: 0.0%

------------------------------------------------------------

[2025-04-14 18:31